parsing equations from strings sandbox

In [1]:
'du/dx^2y^3 * (a+b)'

'du/dx^2y^3 * (a+b)'

In [8]:
eps = 1
w =1 
colloc_weight = 1
def f(x):
    return lambda u_loc, u_bas, x, x_loc:  (u_bas([1,0],0)-eps*u_bas([0,2],0)
                                                -(u_bas([0,1],0)*u_loc([0,1],1)+
                                                u_loc([0,1],0)*u_bas([0,1],1)+

                                                u_loc([0,0],0)*u_bas([0,2],1)+
                                                u_bas([0,0],0)*u_loc([0,2],1))
                                            ) * w**2*colloc_weight

string = "f(0)"
eval(string)

<function __main__.f.<locals>.<lambda>(u_loc, u_bas, x, x_loc)>

In [9]:
# lambda u_loc, u_bas, x, x_loc:  (u_bas([1,0],0)-eps*u_bas([0,2],0)
#                                                 -(u_bas([0,1],0)*u_loc([0,1],1)+
#                                                 u_loc([0,1],0)*u_bas([0,1],1)+

#                                                 u_loc([0,0],0)*u_bas([0,2],1)+
#                                                 u_bas([0,0],0)*u_loc([0,2],1))
#                                             ) * w**2*colloc_weight


string = "d/dt(u0) - eps * d/dx^2(u0) - div(u0 * d/dx(u1))"
#must be result 
string = string.replace('dx', 'dx1')
string = string.replace('dt', 'dx0')

In [11]:
string.split(" ")

['d/dx0(u0)', '-', 'eps', '*', 'd/dx1^2(u0)', '-', 'div(u0', '*', 'd/dx1(u1))']

In [ ]:
'd/dx0(u0)'

In [12]:
eval('f(0)')

0